```
using Microsoft.Extensions.DependencyInjection;

```
```
services.AddSingleton<DatabaseNotificationService>();
var notificationService = app.ApplicationServices.GetService<DatabaseNotificationService>();
            notificationService.SubscribeTableDependency("tb__Log_Order_OrderMaster"); //use the table name in the database

```

```
namespace LOGISTIC_BACKEND.Services
{
    public interface IDatabaseNotificationService
    {
        void SubscribeTableDependency(string tableName);
    }
} //optional use interface and implement it in class

```
```
using System;
using TableDependency.SqlClient;
using TableDependency.SqlClient.Base.Enums;
using TableDependency.SqlClient.Base.EventArgs;
 
namespace LOGISTIC_BACKEND.Services
{
    public class DatabaseNotificationService
    {
        private bool _changeDetectedWithinLastMinute = false;
        private DateTime _lastChangeTime = DateTime.MinValue;
             private SqlTableDependency<LogOrderStatusHistory> _logOrderStatusHistoryTableDependency;
 
        public bool SubscribeTableDependency(string logOrderStatusHistoryTableName)
        {
            try
            {
                string connectionString = Startup.connectiontring;
                if (string.IsNullOrEmpty(connectionString))
                {
                    throw new ArgumentException("Connection string cannot be null or empty.", nameof(connectionString));
                }
                if (string.IsNullOrEmpty(orderTableName) || string.IsNullOrEmpty(logOrderStatusHistoryTableName))
                {
                    throw new ArgumentException("Table names cannot be null or empty.", nameof(orderTableName));
                }
 
              _logOrderStatusHistoryTableDependency = new SqlTableDependency<LogOrderStatusHistory>(connectionString, logOrderStatusHistoryTableName);
                _logOrderStatusHistoryTableDependency.OnChanged += (sender, e) => LogOrderStatusHistoryTableDependency_OnChanged(sender, e);
                _logOrderStatusHistoryTableDependency.OnError += TableDependency_OnError;
                _logOrderStatusHistoryTableDependency.Start();
 
                // Initially, there's no change detected
                return false;
            }
            catch (Exception ex)
            {
                Console.WriteLine($"Error subscribing to table dependency: {ex.Message}");
                return false;
            }
        }
 
        private void LogOrderStatusHistoryTableDependency_OnChanged(object sender, RecordChangedEventArgs<LogOrderStatusHistory> e)
        {
            if (e.ChangeType == ChangeType.Insert)
            {
                Console.Beep();
                // Update the timestamp first
                _lastChangeTime = DateTime.Now;
 
                _changeDetectedWithinLastMinute = (DateTime.Now - _lastChangeTime) <= TimeSpan.FromMinutes(30);
            }
        }
 
        private void TableDependency_OnError(object sender, ErrorEventArgs e)
        {
            Console.WriteLine($"Error on table dependency: {e.Error.Message}");
            // Handle the error here, you might want to log it or take other actions
        }
    }
 
    // Define the class for LogOrderStatusHistory to match your database schema
    
 
    public class LogOrderStatusHistory //same as in the database
    {
        public int UkId { get; set; }
        public int OrderCode { get; set; }
        public int PrevStatus { get; set; }
        public int CurrentStatus { get; set; }
        public string Remarks { get; set; }
        public int StatusReason { get; set; }
        public string ReasonRemarks { get; set; }
        public int UserModuleType { get; set; }
        public string MasterUserID { get; set; }
        public string GUID { get; set; }
        public bool Active { get; set; }
        public DateTime CreateDate { get; set; }
        public int CreateBy { get; set; }
        public int CreatedByModule { get; set; }
        public DateTime ModifiedDate { get; set; }
        public int ModifiedBy { get; set; }
        public int ModifiedByModule { get; set; }
    }

}
 
 

```